<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myProspects_200DMA_RSI14_Signals.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import datetime
import pytz
tz_NY = pytz.timezone('Asia/Kolkata')
datetime_NY = datetime.datetime.now(tz_NY)
print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

Run date time (IST): 2025-11-02 13:36:42


In [2]:
!pip install ta
#!pip install talib-binary

  Preparing metadata (setup.py) ... done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29412 sha256=cbf72e1cd215742f56b5033f60ace6cef91534c06d3329bf51642763a4a87924
  Stored in directory: /root/.cache/pip/wheels/5c/a1/5f/c6b85a7d9452057be4ce68a8e45d77ba34234a6d46581777c6
Successfully built ta


In [3]:
import pandas as pd
import numpy as np
#from google.colab import drive
#drive.mount('/content/drive')

#mypf = pd.read_csv('/content/drive/My Drive/data/stocks/myProspectsScrips.csv')

path_mypf = 'https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/'

mypf = pd.read_csv(path_mypf + 'myProspectsScrips.csv')

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df


pps_df = get_mypps_df()
scr_df = get_myscreen_df()

df_stocks = pd.merge(pps_df, scr_df, on = 'Symbol')

df_stocks['Conviction'] = df_stocks['Conviction'] + '-' + df_stocks['CapType']

stock_list = list(df_stocks[df_stocks['Conviction'].isin(['X-LC','H-LC'])]['Symbol'].values)

all_prospects = stock_list

#print(all_prospects)

list_x40 = list(pps_df[pps_df['MBQ'].str.contains('X40', na=False)]['Symbol'].values)
list_x5k = list(pps_df[pps_df['MBQ'].str.contains('X5K', na=False)]['Symbol'].values)
list_x40n = list(pps_df[pps_df['MBQ'].str.contains('X40N', na=False)]['Symbol'].values)
#len(set(list_x40 + list_x5k) - set( list_x40n))
stock_list = list(set(list_x40 + list_x5k) - set( list_x40n))
#print(stock_list)

#mypf = mypf[mypf.Forecast.notnull()]
#stock_ids = mypf['Symbol'].unique()

#all_prospects.sort()
#stock_ids
exclude = ['SANOFI']

#stock_n100 = mypf[~mypf['Symbol'].isin(exclude) ]['Symbol']
stock_list = list(set(list(stock_list)) - set(exclude))
all_prospects = stock_list
len(all_prospects)

85

In [4]:
# import necessary libraries

%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from tabulate import tabulate
import warnings
warnings.filterwarnings('ignore')
import pandas_datareader.data as web
import yfinance as yfin
import ta

In [5]:
# set start and end date
import datetime
start_date = (datetime.date.today() + datetime.timedelta(days=-365*5)).strftime('%Y-%m-%d')
start1_date = (datetime.date.today() + datetime.timedelta(days=-1*365)).strftime('%Y-%m-%d')
end_date = (datetime.date.today() + datetime.timedelta(days=0)).strftime('%Y-%m-%d')

In [6]:
# function to compute stock attributes
def get_common_features():
  def stock_prec_dev(stock_symbol):
      #stock_symbol = 'ULTRACEMCO.NS'
      short_window = 20
      #mid_window = 50
      long_window = 200
      moving_avg = 'WMA'

      # import the closing price data of the stock for the aforementioned period of time in Pandas dataframe
      start = datetime.datetime(*map(int, start_date.split('-')))
      end = datetime.datetime(*map(int, end_date.split('-')))
      stock_df = yfin.Ticker(stock_symbol).history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]
      stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
      stock_df['200_DMA'] = round(stock_df['Close'].rolling(window = long_window, min_periods = 1).mean(),0)
      stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_DMA'])*100/stock_df['200_DMA'],2)
      stock_df.dropna(axis = 0, inplace = True) # remove any null rows

      stock_df['Stock'] = stock_symbol

      stock_df['Close'] = round(stock_df['Close'],2)
      stock_df['Max'] = round(max(stock_df['Close']),2)
      stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start1_date)]
      stock_df['Min'] = round(min(stock_df['Close']),2)
      stock_df['Gained%'] = round((stock_df['Close'] - stock_df['Min'])*100/stock_df['Min'],2)
      stock_df['ATH%'] = round((stock_df['Max'] - stock_df['Close'])*100/stock_df['Close'],2)
      stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()

      #stock_df['RSI_14'] = ta.momentum.rsi(stock_df['Close'], window=14)
      #stock_df.rename(columns={'RSI_14':'RSI'}, inplace=True)
      stock_df['Close'] = round(stock_df['Close'],0)
      stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
      stock_df.drop(['Open', 'Low', 'High', 'Volume'
                    ], axis=1, inplace=True)

      return stock_df

  df_prec_dev = pd.DataFrame()

  for stock_id in all_prospects:
      #print(stock_id)
      stock_id = stock_id.upper() + '.NS'
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      tmp = tmp.tail(1)
      #df_prec_dev = df_prec_dev.append(tmp)
      df_prec_dev = pd.concat([df_prec_dev, tmp])
  return df_prec_dev, stock_prec_dev

df_common_features, stock_prec_dev = get_common_features()


def get_stock_buy_sell(stock_symbol):

    df_prec_dev = stock_prec_dev(stock_symbol)
    df_prec_dev['Signal'] = 0.0
    df_prec_dev['Signal'] = np.where(((df_prec_dev['Close'] < df_prec_dev['200_DMA']) & (df_prec_dev['RSI_14'] < 30) ), 1.0, df_prec_dev['Signal'])

    df_prec_dev = df_prec_dev[df_prec_dev['Signal'] != 0]

    return df_prec_dev.tail(1)

def get_daily_buy_sell():
    df_buy_sell = pd.DataFrame()
    for stock_id in all_prospects:
        df_tmp = get_stock_buy_sell(stock_id+'.NS')
        if not df_stocks[(df_stocks['Symbol']) == stock_id.upper()].empty:
            df_tmp['Conviction'] = df_stocks[(df_stocks['Symbol']) == stock_id.upper()]['Conviction'].values[0]
            df_tmp['InFolio'] = df_stocks[(df_stocks['Symbol']) == stock_id.upper()]['InFolio'].values[0]
            df_tmp['MBQ'] = df_stocks[(df_stocks['Symbol']) == stock_id.upper()]['MBQ'].values[0]
        else:
            # Handle the case where no matching rows are found
            # You might want to skip this stock or assign default values
            print(f"Warning: No matching rows found for stock_id: {stock_id}")
            continue  # Skip to the next stock_id
        #df_buy_sell = df_buy_sell.append(df_tmp)
        df_buy_sell = pd.concat([df_buy_sell, df_tmp])


    df_buy_sell['Signal'] = df_buy_sell['Signal'].apply(lambda x: 'Buy' if x >= 1 else 'Sell')
    return df_buy_sell.sort_index()

df_daily_buy_sell = get_daily_buy_sell()

In [7]:
df_daily_buy_sell.index = df_daily_buy_sell.index.strftime('%Y-%m-%d')
df_daily_buy_sell[df_daily_buy_sell['Signal'] == 'Buy'].tail(30)

,Close,200_DMA,Dev%_200,Stock,Max,Min,Gained%,ATH%,RSI_14,Signal,Conviction,InFolio,MBQ
Date,,,,,,,,,,,,,
2025-06-20,2944.0,3388.0,-13.11,AKZOINDIA.NS,4210.48,2943.85,0.00,43.03,25.0,Buy,M-SC,NaN,OX40/XR
2025-07-29,1112.0,1142.0,-2.64,NESTLEIND.NS,1363.25,1063.64,4.53,22.61,28.0,Buy,X-LC,1.0,XY25/X40
2025-08-01,1441.0,1662.0,-13.32,HCLTECH.NS,1922.01,1337.19,7.73,33.42,23.0,Buy,X-LC,1.0,X40/X5K
2025-08-01,44.0,50.0,-11.49,IRB.NS,76.32,40.84,8.37,72.44,29.0,Buy,M-SC,NaN,X5K
2025-08-07,169.0,181.0,-6.36,GAIL.NS,230.86,152.29,11.29,36.21,29.0,Buy,L-LC,NaN,X5K
2025-08-08,2178.0,2455.0,-11.28,ADANIENT.NS,4159.41,2094.93,3.97,90.97,20.0,Buy,M-LC,NaN,X5K
2025-08-08,1953.0,1972.0,-0.98,KOTAKBANK.NS,2266.19,1685.91,15.82,16.05,29.0,Buy,X-LC,NaN,X40/X5K
2025-08-08,1058.0,1114.0,-5.02,AXISBANK.NS,1316.18,947.29,11.70,24.39,27.0,Buy,X-LC,NaN,X40/X5K
2025-08-12,2589.0,2669.0,-3.01,GLAXO.NS,3466.20,1931.02,34.06,33.90,23.0,Buy,X-MC,NaN,X40


In [8]:
#tmp_df = df_daily_buy_sell.reset_index()
#tmp_df.to_csv('/content/drive/My Drive/data/stocks/myProspects-Buy-Sell-Signals.csv', index=False)

In [9]:
#stock_symbol = 'INFY.NS'
#plot_stock_buy_sell(stock_symbol)